### **In this notebook. we're going to cover some of the most fundamental concepts of tensors using TensorFlow**

**Most specifically, we're going to cover:**
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Excercises

## **Introduction to Tensors**

### **Creating tensors with tf.constant()**

In [130]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.10.0


In [131]:
# Create tensors with tf.contant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [132]:
# Check the number of dimensions of a tensor
scalar.ndim

0

In [133]:
# Creace a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [134]:
# Check the number of dimensions of a vector
vector.ndim

1

In [135]:
# Create a matrix
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [136]:
# Check the number of dimensions of a matrix
matrix.ndim

2

In [137]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [138]:
# Check the number of dimensions of an another_matrix
another_matrix.ndim

2

In [139]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [140]:
# Check the number of dimensions of a tensor
tensor.ndim

3

**What we've created so far:**

* **Scalar:** a single number
* **Vector:** a number with direction
* **Matrix:** a 2-dimentional array of numbers
* **Tensor:** an n-dimentional array of numbers

### **Creating tensors with tf.Varable()**

In [141]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [142]:
# Let's try change one of the elements in changeable tensor
# changeable_tensor[0] = 7 

# Error Message: 'ResourceVariable' object does not support item assignment

In [143]:
# How about we try .assign
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [144]:
# Let's try change the unchangable tensor
# unchangeable_tensor[0].assign(7) 

# Error Message: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑**Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### **Creating random tensors**

Random tensors are tensors of some abitrary size which contain random numbers.

In [145]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### **Shuffle the order of elements in a tensors**

In [146]:
# Shuffle a tensor (valuable when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                          [3, 4],
                          [2, 5]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [147]:
# Shuffle non-shuffled tensor
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

🔑**Note:** If we want our shuffled tensors to be reproducable, we've got to use the global level random seed as well as the operation level random seed.

### **Other ways to make tensors**

In [148]:
# Create a tenson of all ones
tf.ones([3,5])

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>

In [149]:
# Create a tenson of all ones
tf.zeros([3,4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### **Turn NumPy arrays into tensors**

The main difference between NumPy arrays and TensorFlow tensors it that tensors can be run on a GPU (much faster for numerical computing).

In [150]:
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non capital for vector

import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [151]:
# You can also turn NumPy arrays into tensors
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(3,8))
A, B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]])>)

### **Getting information from tensors**

When dealing with tensors you probably want to be aware of the following attributes:
| **Attribute** | **Meaning** | **Code** |
| :------------: | :-------:  | :-------:  |
|Shape | The lenght (number of elements) of each of the dimensions of a tensor. | tensor.shape |
|Rank | The number of thensor dimesions. A scalar has rank 0, a vector has rank 1, a tensor has rank n. | tensor.ndim |
|Axis or dimension | A particular dimension of a tensor. | tensor[0], tensor[:, 1], ... |
|Size | A total number of items in the tensor. | tf.size(tensor) |

In [152]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 2, 3, 4])
rank_4_tensor

<tf.Tensor: shape=(2, 2, 3, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]]], dtype=float32)>

In [153]:
# Get various attributes of a tensor
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tenson:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in a tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tenson: (2, 2, 3, 4)
Elements along the 0 axis: 2
Elements along the last axis: 4
Total number of elements in a tensor: 48


### **Indexing tensors**

Tensors can be indexed just like python lists.

In [154]:
# Get the first two elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [155]:
rank_4_tensor.shape

TensorShape([2, 2, 3, 4])

In [156]:
# Get the first element from each dimension except for the third one
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 3, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.]]]], dtype=float32)>

In [157]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7], 
                             [3, 4]])
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [158]:
# Get the last item of each row or rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

### **Manipulating tensors (tensor operations)**

**Adding dimensions**

In [159]:
# Add in extra dimension to the rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # "..." means every axis before
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [160]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

**Basic operations**  
`+`, `-`, `*`, `/`

In [161]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], 
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [162]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [163]:
# We can use tensorflow built-in function too
tf.subtract(tensor, 10), tf.add(tensor, 10), tf.multiply(tensor, 10), tf.divide(tensor, 10)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[ 0, -3],
        [-7, -6]])>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[20, 17],
        [13, 14]])>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [ 30,  40]])>,
 <tf.Tensor: shape=(2, 2), dtype=float64, numpy=
 array([[1. , 0.7],
        [0.3, 0.4]])>)

🔑**Note:** Manipulating tensors with built-in tensorflow functions is way faster than performing a basic operation.

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.  

There are two rules our tensors (matrices) need to fulfill if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimension

In [164]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [165]:
# Matrix multiplication in tensorflow
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [166]:
# Create a tensor (3, 2)
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
                 
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [167]:
# tf.matmul(X, Y)
# Error Message: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

X.shape, Y.shape
# Metrices need to have the same inner dimension, in this case they don't (3 != 2)

(TensorShape([3, 2]), TensorShape([3, 2]))

In [168]:
# Let's change the shape of the Y matrix
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]])>

In [169]:
# Try the multiplication again
tf.matmul(X, tf.reshape(Y, shape=(2, 3))) # outer dimensions are (3, 3), so is the output

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [170]:
# Try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y) # outer dimensions are (2, 2), so is the output

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]])>

In [171]:
# You can also transpose tensors (swap the axises)
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]])>)

**The dot product**

Matrix multiplication is also refered as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [172]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [173]:
# Perform the dot product on X and Y
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

In [174]:
print('Normal Y:')
print(Y, '\n')
print('Y reshaped to (2, 3):')
print(tf.reshape(Y, shape=(2, 3)))
print(Y, '\n')
print('Y transposed:')
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


🔑**Note:** Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy matrix multiplication rules.

### **Changing the datatype of the tensor**

In [175]:
# Create a new tensor with a default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [176]:
C = tf.constant([7, 10])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>, tf.int32)

In [177]:
# Change from float32 to float16 (raduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [178]:
# Change from int32 to float16
E = tf.cast(C, dtype=tf.float16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>,
 tf.float16)

### **Aggregating tensors**

Aggregating tensors = condensing them from multiple values to a smaller amutny of values.

In [179]:
# Getting the absolute values
D = tf.constant([-7, -10])
D, tf.abs(D)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>)

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean
* Get the sum

In [180]:
# Create a random tensor with values between 0 and 100 and size of 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([17,  2, 13, 75, 37, 91, 70, 83, 36, 40, 80, 68, 53, 86,  7, 53, 72,
       22, 45,  5, 21, 60, 70, 84,  0, 83, 38, 21, 52, 65, 23, 35, 11, 50,
       81, 24, 82, 42, 82, 74, 89, 61, 94, 78,  1, 89,  4, 64, 66, 71])>

In [181]:
# Basic attributes of a tensor
tf.size(E).numpy(), E.shape, E.ndim

(50, TensorShape([50]), 1)

In [182]:
# Find minimum, maximum, mean and sum of a tensor
print('Minimum:')
print(tf.reduce_min(E))
print('Maximum:')
print(tf.reduce_max(E))
print('Mean:')
print(tf.reduce_mean(E))
print('Sum:')
print(tf.reduce_sum(E))

Minimum:
tf.Tensor(0, shape=(), dtype=int32)
Maximum:
tf.Tensor(94, shape=(), dtype=int32)
Mean:
tf.Tensor(51, shape=(), dtype=int32)
Sum:
tf.Tensor(2570, shape=(), dtype=int32)


In [183]:
# Find variance and standard deviation of a tensor
print('Variance:')
print(tf.math.reduce_variance(tf.cast(E, dtype=tf.float16)))
print('Standard Deviation:')
print(tf.math.reduce_std(tf.cast(E, dtype=tf.float16))) 

Variance:
tf.Tensor(841.0, shape=(), dtype=float16)
Standard Deviation:
tf.Tensor(29.0, shape=(), dtype=float16)


### **Find the positional minimum and maximum**

In [184]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [185]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [186]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [187]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

### **Squeezing a tensor (removing all single dimensions)**

In [188]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G, G.shape

(<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
 array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
            0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
            0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
            0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
            0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
            0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
            0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
            0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
            0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
            0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
       dtype=float32)>,
 TensorShape([1, 1, 1, 1, 50]))

In [189]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### **One-hot encoding tensors**

In [198]:
# Create a list of indicies
some_list = [0, 1, 2, 3, 2, 1, 0] # could be red, green, blue, purple

# One the encode our indicies
tf.one_hot(some_list, depth=4, dtype=tf.float16)

<tf.Tensor: shape=(7, 4), dtype=float16, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float16)>

### **Squaring, log, square root**

In [199]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [208]:
# Find minimum, maximum, mean and sum of a tensor
print('Square:')
print(tf.square(H))
print('Square root:')
print(tf.sqrt(tf.cast(H, dtype=tf.float16)))
print('Log:')
print(tf.math.log(tf.cast(H, dtype=tf.float16)))

Square:
tf.Tensor([ 1  4  9 16 25 36 49 64 81], shape=(9,), dtype=int32)
Square root:
tf.Tensor([1.    1.414 1.732 2.    2.236 2.45  2.646 2.828 3.   ], shape=(9,), dtype=float16)
Log:
tf.Tensor([0.     0.6934 1.099  1.387  1.609  1.792  1.946  2.08   2.197 ], shape=(9,), dtype=float16)


### **Tensors in NumPy**

TensorFlow interact beutifully with NumPy arrays.

In [210]:
# Creare a tensor directly from NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [214]:
# Convert our tensor back to our NumPy array
np.array(J), J.numpy()

(array([ 3.,  7., 10.]), array([ 3.,  7., 10.]))

In [220]:
# Default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)